In [16]:
from kafka import KafkaProducer
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

import tkinter as tk

import requests
import json
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

In [17]:
#Retourne le code source de l'URL qui est passé en paramètre
def get_source(url):
    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

In [18]:
#Retourne un dataFrame qui contient le contenu du flux RSS passé en paramètre
def get_feed(url):   
    response = get_source(url)
    
    df = pd.DataFrame(columns = ['title', 'pubDate', 'link', 'description'])

    with response as r:
        items = r.html.find("item", first=False)

        for item in items:        

            title = item.find('title', first=True).text
            pubDate = item.find('pubDate', first=True).text
            link = item.find('guid', first=True).text
            description = item.find('description', first=True).text


            row = {'title': title, 'pubDate': pubDate, 'link': link, 'description': description}
            df = df.append(row, ignore_index=True)

    return df

In [19]:
#Permet à l'utilisateur de sélectionner les sites internet qu'il veut consulter parmi une liste prédéfinie
#Retourne une liste qui contient ces sites 
def ouvrir_fenetre():
    sites = {
        "Le Monde": "https://www.lemonde.fr/rss/une.xml",
        "OVH Cloud": "https://blog.ovhcloud.com/feed/",
        "Actu IA": "https://www.actuia.com/feed/",
        "20 Minutes": "https://www.20minutes.fr/feeds/rss-une.xml",
        "France 24":"https://www.france24.com/fr/rss"
    }

    def ajouter_site():
        selection.clear()
        for site in listebox.curselection():
            nom_site = listebox.get(site)
            selection.append(sites[nom_site])
        fenetre.quit()

    fenetre = tk.Tk()

    listebox = tk.Listbox(fenetre, selectmode=tk.MULTIPLE)
    for site in sites:
        listebox.insert(tk.END, site)

    bouton_ajouter = tk.Button(fenetre, text="Ajouter", command=ajouter_site)

    selection = []

    listebox.grid(row=0, column=0)
    bouton_ajouter.grid(row=1, column=0)

    fenetre.mainloop()

    return selection

In [20]:
#Appel de la fonction
urls=ouvrir_fenetre()
urls

['https://www.lemonde.fr/rss/une.xml',
 'https://blog.ovhcloud.com/feed/',
 'https://www.actuia.com/feed/',
 'https://www.20minutes.fr/feeds/rss-une.xml',
 'https://www.france24.com/fr/rss']

In [25]:
# Création du dataFrame en fonction des sites sélectionnés
dfs = []
for url in urls:
    df = get_feed(url)
    dfs.append(df)

# Fusion des dataFrame en un seul
data = pd.concat(dfs, ignore_index=True)

c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\anton\AppData\Local\Temp\ipykernel_18132\664555981.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\anton\AppData\Local\Temp\ipykernel_18132\664555981.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append

In [26]:
#Nettoyage des données 
data['title'] = data['title'].apply(lambda x: x.replace('ê', 'e'))
data['title'] = data['title'].apply(lambda x: x.replace("'", ""))
data['description'] = data['description'].apply(lambda x: x.replace("'", ""))
for i in range(len(data)):
    data['description'] = data['description'].apply(lambda x: x.replace('ê', 'e'))
    if '[' in data.loc[i,'title'] and ']' in data.loc[i,'title']:
        data.loc[i,'title'] = data.loc[i,'title'].split('<![CDATA[')[1].split(']')[0] 
    if '[' in data.loc[i,'description'] and ']' in data.loc[i,'description']:
        data.loc[i,'description'] = data.loc[i,'description'].split('<![CDATA[')[1].split(']')[0] 
    else:
        data.loc[i,'title'] = data.loc[i,'title']
        data.loc[i,'description'] = data.loc[i,'description']

In [29]:
#Classement des articles par ordre décroissant (plus récents en premier)
data['pubDate'] = pd.to_datetime(data['pubDate'], format='%a, %d %b %Y %H:%M:%S %z')
data = data.sort_values('pubDate', ascending=False)
data = data.reset_index(drop=True)

ValueError: time data 'Thu, 06 Apr 2023 08:49:41 GMT' does not match format '%a, %d %b %Y %H:%M:%S %z' (match)

In [ ]:
data

,title,pubDate,link,description
0,,"Thu, 06 Apr 2023 20:48:43 +0200",https://www.lemonde.fr/politique/article/2023/...,
1,,"Thu, 06 Apr 2023 19:14:25 +0200",https://www.lemonde.fr/international/live/2023...,
2,,"Thu, 06 Apr 2023 16:37:49 +0200",https://www.lemonde.fr/societe/article/2023/04...,
3,,"Thu, 06 Apr 2023 15:57:42 +0200",https://www.lemonde.fr/international/article/2...,
4,,"Thu, 06 Apr 2023 17:00:05 +0200",https://www.lemonde.fr/planete/live/2023/04/06...,
...,...,...,...,...
90,"Le portier, la playmate, la star du X et les é...","Wed, 05 Apr 2023 02:27:05 GMT",4971c82e-d359-11ed-95c4-005056bf762b,Inculpé dans une affaire de versements occulte...
91,"Emmanuel Macron en Chine : ""quelques opportuni...","Tue, 04 Apr 2023 15:17:20 GMT",73fd4ee8-d2d6-11ed-9fe7-005056bf762b,Emmanuel Macron se rend en Chine mercredi pour...
92,Pourquoi les États-Unis veulent élever la Chin...,"Tue, 04 Apr 2023 15:51:23 GMT",5f85f652-d2f6-11ed-9b28-005056bf762b,La Chambre des représentants des États-Unis a ...
93,"Pour Lecornu, ""il ny a rien de trop"" dans les ...","Wed, 05 Apr 2023 08:15:19 GMT",70e762c4-d386-11ed-bfa7-005056a97652,"Le ministre des Armées, Sébastien Lecornu, a d..."


In [ ]:
# Configure Kafka
bootstrap_servers = ['localhost:9092']
topic = 'your_kafka_topic'
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)

# Configure Cassandra
contact_points = ['localhost']
auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')
cluster = Cluster(contact_points=contact_points, auth_provider=auth_provider)
session = cluster.connect()


# Envoie du dataFrame vers Kafka
serialized_df = data.to_json().encode('utf-8')  # encode the JSON string to bytes
producer.send(topic, value=serialized_df)

# Le dataFrame est ensuite stocké dans la base de données Cassandra
session.execute("CREATE KEYSPACE IF NOT EXISTS your_keyspace WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}")
session.execute("USE your_keyspace")
session.execute("CREATE TABLE IF NOT EXISTS RSS (title text, pubdate text, link text, description text, PRIMARY KEY(pubdate, title)) WITH CLUSTERING ORDER BY (title DESC)")

for _, row in data.iterrows():
    session.execute(f"INSERT INTO RSS (title, pubdate, link, description) VALUES ('{row['title']}', '{row['pubDate']}', '{row['link']}', '{row['description']}')")

In [ ]:
rows = session.execute("SELECT * FROM RSS")
for row in rows:
    print(row.link)

https://www.20minutes.fr/serie/4031569-20230406-transatlantique-histoire-palpitante-reseau-justes-injustement-meconnus
https://www.lemonde.fr/afrique/article/2023/04/06/le-president-tunisien-kais-saied-dit-non-au-fmi_6168564_3212.html
https://www.lemonde.fr/economie/article/2023/04/06/crise-au-canard-enchaine-l-ancien-redacteur-en-chef-claude-angeli-intervient_6168574_3234.html
https://www.lemonde.fr/economie/article/2023/04/06/carburants-la-justice-suspend-les-requisitions-de-grevistes-de-totalenergies-de-gonfreville_6168559_3234.html
ea3ddd64-d4a9-11ed-98bb-005056a97652
https://www.20minutes.fr/planete/4031491-20230406-eau-potable-francaise-contaminee-residus-fongicide-interdit
dd5fe83c-d3d5-11ed-b010-005056a97652
https://www.20minutes.fr/planete/4030986-20230406-demande-phenomenale-recuperateurs-eau-pluie-arme-contre-secheresse
c9bc4440-d3c3-11ed-8eb7-005056a97652
5f85f652-d2f6-11ed-9b28-005056bf762b
https://www.20minutes.fr/monde/ukraine/4031434-20230406-guerre-ukraine-direct-macro

In [14]:
#Ferme la connection
def close_connection():
    producer.flush()
    producer.close()
    session.shutdown()
    cluster.shutdown()
    producer.flush()
    producer.close()
    session.shutdown()
    cluster.shutdown()

In [15]:
#Suppression des données dans la basse de données
def delete_data():
    session.execute("DESCRIBE KEYSPACES;")
    session.execute("USE your_keyspace;")
    session.execute("DESCRIBE TABLES;")
    session.execute("TRUNCATE RSS")

In [16]:
delete_data()